In [1]:
from keras import applications
from evec_scan.tools import evectools as evt
import pandas as pd
import os
import numpy as np

In [2]:
df = pd.DataFrame()
instance_dir = 'evec_scan/instances/test1'
vid_file = 'vid3.mp4'
instance_id = 'test1'
group_size = 2
vidpath = 'evec_scan/instances/test1/vid3.mp4'
description_models = ['moondream'] #['moondream', 'llava-llama3']
description_embedding_models = ['mxbai-embed-large'] #['mxbai-embed-large','nomic-embed-text', 'all-minilm']
#######################################################################################################


In [4]:
video_instance = evt.FrameExtractor(vidpath=vidpath)
frame_group = video_instance.get_random_frame_group(size=group_size)
frame_group_instance = video_instance.FrameGroup(frame_extractor=video_instance, group_set=frame_group)
frames_output_path = os.path.join(instance_dir, 'frames')
frame_path_list = frame_group_instance.save_group(output_dir=frames_output_path, group_name=instance_id)
df['frame_path'] = frame_path_list
df.index = list(range(len(frame_path_list)))
########################################################################################################


files were succesfully saved


In [5]:
for desc_model in description_models: 
    descriptions_per_model = []
    description_columns = []
    for frame_path in df['frame_path']:
        descriptor = evt.ImgDescriptor(model=desc_model,frame_path=frame_path)
        description = descriptor.get_description()
        descriptions_per_model.append(description)
    df[f'{desc_model}_desc'] = descriptions_per_model
    description_columns.append(f'{desc_model}_desc')
#######################################################################################################3


In [6]:
df

,frame_path,moondream_desc
0,evec_scan/instances/test1\frames\test1_6208.jpg,"\nThe image depicts a city street at night, wi..."
1,evec_scan/instances/test1\frames\test1_7447.jpg,"\nThe image depicts a city street at night, wi..."


In [7]:
for dir in description_columns:
    description_evec_output_dir = os.path.join(instance_dir, 'desc_evecs', dir)
    os.makedirs(description_evec_output_dir)
    for text_evec_model in description_embedding_models:
        description_evec_arr_per_model_per_desc_model = np.empty(group_size, dtype=object)
        i = 0
        for description in df[dir]:
            text_evec_scanner = evt.TextEvecScanner(model=text_evec_model, text=description)
            text_evec = text_evec_scanner.get_evec()
            description_evec_arr_per_model_per_desc_model[i]=text_evec
        output_path = os.path.join(description_evec_output_dir,f'{text_evec_model}.npy')
        np.save(file=output_path, arr=description_evec_arr_per_model_per_desc_model)




